<a href="https://colab.research.google.com/github/nubialmeida/Projeto-Pyspark/blob/main/Projeto_Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalação de bibliotecas

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=11faca74d09eac86cb82afe7c6091a4013ed337c860dc8f7dc944b57a0a235f6
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


##Importação de bibliotecas

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
from pyspark.sql.types import IntegerType, FloatType, DoubleType

##Montando drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Iniciando a sessão

In [ ]:
spark = (SparkSession.builder
                     .master('local')
                     .appName('spotify')
                     .config('spark.ui.port', '4050')
                     .getOrCreate())

##Extração dos dados(Extract)

In [ ]:
df = (spark.read
           .format('csv')
           .option('delimiter', ',')
           .option('header', 'true')
           .option('inferschema', 'true')
           .option('encoding', 'utf-8')
           .option('escape','"')
           .load('/content/drive/MyDrive/TI/spotify.csv')
           )

In [ ]:
spark

##Pré Análise

In [ ]:
df.printSchema() #Verificando os tipos do DF

root
 |-- _c0: integer (nullable = true)
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)



In [ ]:
print(f'({df.count()}, {len(df.columns)})')#Verificando a contagem de linhas e colunas

(18835, 16)


In [ ]:
df.show(truncate=False) #Verificando o DF

+---+--------------------------+---------------+----------------+----------------------+-----------------------+---------------------+---------------------+---+----------------------+-----------------------+----------+----------------------+-------+--------------+---------------------+
|_c0|song_name                 |song_popularity|song_duration_ms|acousticness          |danceability           |energy               |instrumentalness     |key|liveness              |loudness               |audio_mode|speechiness           |tempo  |time_signature|audio_valence        |
+---+--------------------------+---------------+----------------+----------------------+-----------------------+---------------------+---------------------+---+----------------------+-----------------------+----------+----------------------+-------+--------------+---------------------+
|0  |Boulevard of Broken Dreams|73             |262333          |0.005520000000000001kg|0.496mol/L             |682                  |2.94e

In [ ]:
#Quantidade de registros do dataframe
qtd_registros = df.count()

In [ ]:
df.drop(F.col('_c0')).filter(F.col('song_name') == 'TEST DRIVE').drop_duplicates().show(truncate=False)

+----------+---------------+----------------+------------+-----------------------+------+----------------+---+--------+--------+----------+-----------+---------------------+--------------+-------------+
|song_name |song_popularity|song_duration_ms|acousticness|danceability           |energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo                |time_signature|audio_valence|
+----------+---------------+----------------+------------+-----------------------+------+----------------+---+--------+--------+----------+-----------+---------------------+--------------+-------------+
|TEST DRIVE|71             |179423          |0.71kg      |0.6459999999999999mol/L|648   |316             |3  |108     |-8.602  |1         |372        |7.500.399.999.999.990|4             |466          |
+----------+---------------+----------------+------------+-----------------------+------+----------------+---+--------+--------+----------+-----------+---------------------+--------------+

In [ ]:
df.select('song_name').distinct().count()

13070

In [ ]:
#Verificando se existe algum duplicado
qtd_sem_duplicados = df.drop(F.col('_c0')).drop_duplicates().count()

In [ ]:
total_duplicados = qtd_registros - qtd_sem_duplicados

In [ ]:
print(f'Total de linhas duplicadas: {total_duplicados}')

Total de linhas duplicadas: 3903


In [ ]:
df = df.drop(F.col('_c0'))

In [ ]:
#Removendo valores duplicados
df = df.drop_duplicates()

In [ ]:
df.count()

14932

In [ ]:
df.show(truncate=False)

+----------------------------------------+---------------+----------------+-----------------------+-----------------------+---------------------+---------------------+---+----------------------+-----------------------+----------+-----------+----------------------+--------------+----------------------+
|song_name                               |song_popularity|song_duration_ms|acousticness           |danceability           |energy               |instrumentalness     |key|liveness              |loudness               |audio_mode|speechiness|tempo                 |time_signature|audio_valence         |
+----------------------------------------+---------------+----------------+-----------------------+-----------------------+---------------------+---------------------+---+----------------------+-----------------------+----------+-----------+----------------------+--------------+----------------------+
|Lambada - Original Version 1989         |67             |207466          |0.211kg         

In [ ]:
#Verificando Summary
df.summary().show()

+-------+--------------------+------------------+--------------------+------------+------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+
|summary|           song_name|   song_popularity|    song_duration_ms|acousticness|danceability|            energy|  instrumentalness|               key|          liveness|          loudness|        audio_mode|       speechiness|             tempo|      time_signature|     audio_valence|
+-------+--------------------+------------------+--------------------+------------+------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+
|  count|               14932|             14932|               14932|       14932|       14932|             14932|             14932

In [ ]:
#Criando uma nova tabela com nome acustica e removendo kg da coluna acousticness 
df = df.withColumn('acousticness', F.regexp_replace('acousticness', 'kg', ''))

In [ ]:
df.show()

+--------------------+---------------+----------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+--------------------+
|           song_name|song_popularity|song_duration_ms|        danceability|              energy|    instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|            acustica|
+--------------------+---------------+----------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+--------------------+
|Lambada - Origina...|             67|          207466|0.755999999999999...|                 0.8|                 723|  5|                  47|              -5.158|         1|        329|             11

In [ ]:
# Removendo o mol/L da coluna danceability
df = df.withColumn('danceability', F.regexp_replace('danceability', 'mol/L', ''))

In [ ]:
df.select('song_popularity').summary().show()

+-------+------------------+
|summary|   song_popularity|
+-------+------------------+
|  count|             14932|
|   mean| 48.75768535262206|
| stddev|20.379617796732816|
|    min|                 0|
|    25%|              37.0|
|    50%|              52.0|
|    75%|              64.0|
|    max|           nao_sei|
+-------+------------------+



In [ ]:
#função que remove strings

def remove_strings_inplace(df,palavra, colunas):
    # itera sobre as colunas especificadas e aplica a transformação em cada uma
    for column in colunas:
        df = df.withColumn(column, F.regexp_replace(F.col(column), palavra, ""))
    return df
#chama a função passando de parâmetro o df e a lista criada

In [ ]:
colunas = df.columns

In [ ]:
df = remove_strings_inplace(df,'kg', colunas)

In [ ]:
df = remove_strings_inplace(df,'mol/L', colunas)

In [ ]:
#Verificando os não_sei da coluna song_popularity
df.filter(F.col('instrumentalness').contains('nao')).show()

+--------------------+---------------+----------------+------------+------------------+------+----------------+---+--------+--------------------+----------+-----------+------+--------------+-------------+
|           song_name|song_popularity|song_duration_ms|acousticness|      danceability|energy|instrumentalness|key|liveness|            loudness|audio_mode|speechiness| tempo|time_signature|audio_valence|
+--------------------+---------------+----------------+------------+------------------+------+----------------+---+--------+--------------------+----------+-----------+------+--------------+-------------+
|     Sunny Came Home|             62|          264200|       0.344|0.5539999999999999|  0.58|         nao_sei| 11|     989|               -8.05|         0|        322|167.83|             4|          402|
|Love Will Tear Us...|             60|          223340|       0.951|             0.375|   217|         nao_sei|  3|     107|-12.425.999.999.9...|         1|        344|96.239|     

In [ ]:
df = df.replace('nao_sei', None)
df = df.replace('0.nao_sei', None)
df = df.dropna(how='any')

In [ ]:
#Verificando a quantidade de nulos no DF
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|        0|              0|               0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+



In [ ]:
df.select(F.col('instrumentalness')).summary().show()

+-------+------------------+
|summary|  instrumentalness|
+-------+------------------+
|  count|             14919|
|   mean|164.09098902340065|
| stddev|275.64426920379583|
|    min|               0.0|
|    25%|               0.0|
|    50%|           1.14E-5|
|    75%|             223.0|
|    max|               999|
+-------+------------------+



In [ ]:
df.select(F.col('instrumentalness')).orderBy(F.col('instrumentalness').desc()).show(500)

+----------------+
|instrumentalness|
+----------------+
|             999|
|             998|
|             998|
|             998|
|             997|
|             997|
|             996|
|             995|
|             994|
|             994|
|             994|
|             992|
|             992|
|             991|
|             991|
|             991|
|             991|
|              99|
|              99|
|              99|
|              99|
|             989|
|             989|
|             989|
|             989|
|             989|
|             989|
|             988|
|             987|
|             987|
|             986|
|             986|
|             986|
|             986|
|             985|
|             984|
|             984|
|             984|
|             983|
|             983|
|             982|
|             982|
|             982|
|             981|
|             981|
|             979|
|             979|
|             978|
|             978|
|           

In [ ]:
df.select(F.col('instrumentalness')).withColumn('teste', F.col('instrumentalness').cast(DoubleType())).printSchema()

root
 |-- instrumentalness: string (nullable = true)
 |-- teste: double (nullable = true)



In [ ]:
df.printSchema()

root
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)



In [ ]:
#CONVERTER AS COLUNAS PARA INT E FLOAT
df = (df.withColumn('song_popularity', F.col('song_popularity').cast(IntegerType()))
   .withColumn('song_duration_ms', F.round(F.col('song_duration_ms').cast(IntegerType()), 2))
   .withColumn('danceability', F.round(F.col('danceability').cast(FloatType()), 2))
   .withColumn('energy', F.round(F.col('energy').cast(FloatType()), 2))
   .withColumn('instrumentalness', F.round(F.col('instrumentalness').cast(FloatType()), 2))
   .withColumn('key', F.round(F.col('key').cast(FloatType()), 2))
   .withColumn('liveness', F.round(F.col('liveness').cast(FloatType()), 2))
   .withColumn('loudness', F.round(F.col('loudness').cast(FloatType()), 2))
   .withColumn('audio_mode', F.round(F.col('audio_mode').cast(IntegerType()), 2))
   .withColumn('speechiness', F.round(F.col('speechiness').cast(FloatType()), 2))
   .withColumn('tempo', F.round(F.col('tempo').cast(IntegerType()), 2))
   .withColumn('time_signature', F.round(F.col('time_signature').cast(IntegerType()), 2))
   .withColumn('audio_valence', F.round(F.col('audio_valence').cast(FloatType()), 2))
   .withColumn('acustica', F.round(F.col('acustica').cast(FloatType()), 2))
)

In [ ]:
df.show()

+--------------------+---------------+----------------+--------------------+------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|           song_name|song_popularity|song_duration_ms|        acousticness|      danceability|              energy|    instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|
+--------------------+---------------+----------------+--------------------+------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|Lambada - Origina...|             67|          207466|               0.211|0.7559999999999999|                 0.8|                 723|  5|                  47|              -5.158|         1|        329|  

In [ ]:
df.summary().show()

+-------+--------------------+-----------------+--------------------+-------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+--------------------+------------------+
|summary|           song_name|  song_popularity|    song_duration_ms|       acousticness|       danceability|           energy|  instrumentalness|               key|          liveness|          loudness|         audio_mode|      speechiness|             tempo|      time_signature|     audio_valence|
+-------+--------------------+-----------------+--------------------+-------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+-------------------+-----------------+------------------+--------------------+------------------+
|  count|               14919|            14919|               14919|              14919|        

In [ ]:
df.sort(F.col('song_popularity').desc()).show(truncate=False)

+----------------------------------------------+---------------+----------------+-------------------+------------------+---------------------+----------------+---+---------------------+-----------------------+----------+-----------+----------------------+--------------+----------------------+
|song_name                                     |song_popularity|song_duration_ms|acousticness       |danceability      |energy               |instrumentalness|key|liveness             |loudness               |audio_mode|speechiness|tempo                 |time_signature|audio_valence         |
+----------------------------------------------+---------------+----------------+-------------------+------------------+---------------------+----------------+---+---------------------+-----------------------+----------+-----------+----------------------+--------------+----------------------+
|I Love It (& Lil Pump)                        |99             |127946          |0.0114             |0.901            

In [ ]:
df.sort(F.col('danceability').desc()).show(truncate=False)

+---------------------------------+---------------+----------------+--------------------+------------+----------------------+----------------+---+--------+-----------------------+----------+----------------------+-------+--------------+---------------------+
|song_name                        |song_popularity|song_duration_ms|acousticness        |danceability|energy                |instrumentalness|key|liveness|loudness               |audio_mode|speechiness           |tempo  |time_signature|audio_valence        |
+---------------------------------+---------------+----------------+--------------------+------------+----------------------+----------------+---+--------+-----------------------+----------+----------------------+-------+--------------+---------------------+
|Walk Away                        |36             |278653          |0.00317             |0.987       |358                   |464             |9  |943     |-8.677.999.999.999.990 |0         |8.800.000.000.000.000 |117.978|4 

In [ ]:
df.sort(F.col('energy').desc()).show(truncate=False)

+------------------------------+---------------+----------------+---------------------+-------------------+------+---------------------+---+----------------------+-----------------------+----------+-----------+----------------------+--------------+----------------------+
|song_name                     |song_popularity|song_duration_ms|acousticness         |danceability       |energy|instrumentalness     |key|liveness              |loudness               |audio_mode|speechiness|tempo                 |time_signature|audio_valence         |
+------------------------------+---------------+----------------+---------------------+-------------------+------+---------------------+---+----------------------+-----------------------+----------+-----------+----------------------+--------------+----------------------+
|Milkshake                     |27             |196074          |0.0207               |0.767              |999   |887                  |6  |0.22                  |-47.810.000.000.000.0

In [ ]:
df.sort(F.col('tempo').desc()).show(truncate=False)

+-------------------------------------+---------------+----------------+-------------------+------------------+---------------------+---------------------+---+--------+-----------------------+----------+-----------+------+--------------+---------------------+
|song_name                            |song_popularity|song_duration_ms|acousticness       |danceability      |energy               |instrumentalness     |key|liveness|loudness               |audio_mode|speechiness|tempo |time_signature|audio_valence        |
+-------------------------------------+---------------+----------------+-------------------+------------------+---------------------+---------------------+---+--------+-----------------------+----------+-----------+------+--------------+---------------------+
|Paris                                |79             |221520          |0.0243             |0.6459999999999999|644                  |0.0                  |2  |888     |-6.763                 |1         |31         |99.99

In [ ]:
df.sort(F.col('instrumentalness').desc()).show(truncate=False)

+------------------------------------+---------------+----------------+------------+------------------+---------------------+----------------+---+--------+-----------------------+----------+-----------+---------------------+--------------+-------------+
|song_name                           |song_popularity|song_duration_ms|acousticness|danceability      |energy               |instrumentalness|key|liveness|loudness               |audio_mode|speechiness|tempo                |time_signature|audio_valence|
+------------------------------------+---------------+----------------+------------+------------------+---------------------+----------------+---+--------+-----------------------+----------+-----------+---------------------+--------------+-------------+
|Veranito de San Juan                |37             |273800          |0.454       |0.565             |7.829.999.999.999.990|999             |4  |726     |-7.356                 |1         |119        |75.99                |4             

##Conclusão final

####AS MÚSICAS POPULARES SÃO AS MAIS DANÇANTES? Não
####AS MÚSICAS POPULARES SÃO AS COM MAIOR ENERGIA? Não
####AS MÚSICAS MAIS POPULARES POSSUEM MAIOR TEMPO? Não
####AS MÚSICAS COM INSTRUMENTALIDADE SÃO AS MAIS POPULARES? Não